<a href="https://colab.research.google.com/github/EEECD-ondeviceLLM/finetuning/blob/main/test%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EA%B2%80%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 라이브러리 설치 (Colab 환경 호환 버전)
# 의존성 충돌을 피하기 위해 주요 라이브러리 버전을 명시합니다.
# !pip install -U transformers>=4.51.0 torch>=2.7.0 torchaudio>=2.7.0 torchvision trl>=0.17.0 peft bitsandbytes accelerate datasets tqdm
!pip install -U transformers torch torchaudio torchvision trl peft bitsandbytes accelerate datasets tqdm peft



In [ ]:
# 2. 필수 라이브러리 임포트
import os
import re
import torch
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_from_disk, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# prompt: prepared_finetuning_dataset_splits_arrow unzip해줘

!unzip prepared_finetuning_dataset_splits_arrow.zip

Archive:  prepared_finetuning_dataset_splits_arrow.zip
   creating: prepared_finetuning_dataset_splits_arrow/
  inflating: __MACOSX/._prepared_finetuning_dataset_splits_arrow  
  inflating: prepared_finetuning_dataset_splits_arrow/dataset_dict.json  
  inflating: __MACOSX/prepared_finetuning_dataset_splits_arrow/._dataset_dict.json  
   creating: prepared_finetuning_dataset_splits_arrow/test/
  inflating: __MACOSX/prepared_finetuning_dataset_splits_arrow/._test  
   creating: prepared_finetuning_dataset_splits_arrow/train/
  inflating: __MACOSX/prepared_finetuning_dataset_splits_arrow/._train  
   creating: prepared_finetuning_dataset_splits_arrow/validation/
  inflating: __MACOSX/prepared_finetuning_dataset_splits_arrow/._validation  
  inflating: prepared_finetuning_dataset_splits_arrow/test/state.json  
  inflating: __MACOSX/prepared_finetuning_dataset_splits_arrow/test/._state.json  
  inflating: prepared_finetuning_dataset_splits_arrow/test/dataset_info.json  
  inflating: __MACOS

In [ ]:
# 3. 설정값 정의
# model_hub_id   = "totalcream/Qwen3-4B-SFT-DPO-Merged"
model_hub_id   = "Qwen/Qwen3-4B"
# model_hub_id = "totalcream/Qwen3-4B-finetunning-model"
dataset_path   = "prepared_finetuning_dataset_splits_arrow"
batch_size     = 4  # VRAM 사용량에 따라 8, 4, 2 등으로 조정하세요.

print(f"✅ 평가 준비 시작: 모델='{model_hub_id}', 데이터셋='{dataset_path}'")

# 4. 토크나이저 및 모델 로딩
tokenizer = AutoTokenizer.from_pretrained(model_hub_id, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_hub_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

model.eval()

print("✅ 모델 및 토크나이저 로딩 완료")

✅ 평가 준비 시작: 모델='totalcream/Qwen3-4B-finetunning-model', 데이터셋='prepared_finetuning_dataset_splits_arrow'


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# === 2. 메인 코드 실행 (런타임 재시작 후 이 셀만 실행) ===

# --- 2-1. 필수 라이브러리 임포트 ---
import os
import re
import torch
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import PeftModel # LoRA 어댑터 적용을 위한 핵심 클래스

print("✅ 라이브러리 임포트 완료.")

# --- 2-2. 설정값 정의 ---
base_model_id  = "Qwen/Qwen3-4B"
adapter_id     = "totalcream/Qwen3-4B-finetunning-Adapter"
dataset_path   = "prepared_finetuning_dataset_splits_arrow"
batch_size     = 4  # VRAM 사용량에 따라 8, 4, 2 등으로 조정

print(f"\n✅ 평가 준비 시작:")
print(f"  - 베이스 모델: '{base_model_id}'")
print(f"  - LoRA 어댑터: '{adapter_id}'")
print(f"  - 데이터셋: '{dataset_path}'")

# --- 2-3. 베이스 모델과 LoRA 어댑터 로딩 ---
# 1. 토크나이저는 베이스 모델의 것을 사용합니다.
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

# 2. 베이스 모델을 불러옵니다.
print("\n⏳ 베이스 모델을 로딩합니다...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)
print("✅ 베이스 모델 로딩 완료.")

# 3. 로드된 베이스 모델 위에 LoRA 어댑터 레이어를 적용합니다.
print(f"⏳ 베이스 모델에 '{adapter_id}' LoRA 어댑터를 적용합니다...")
model = PeftModel.from_pretrained(model, adapter_id)
print("✅ LoRA 어댑터 적용 완료.")

# # (선택사항) 만약 추론 속도를 더 높이고 싶다면 아래 병합 코드를 사용할 수 있습니다.
# # 단, 더 많은 메모리가 필요할 수 있습니다.
# print("Merging adapter into base model for faster inference...")
# model = model.merge_and_unload()

model.eval() # 평가 모드로 설정
print("\n✅ 최종 모델 준비 완료!")

✅ 라이브러리 임포트 완료.

✅ 평가 준비 시작:
  - 베이스 모델: 'Qwen/Qwen3-4B'
  - LoRA 어댑터: 'totalcream/Qwen3-4B-finetunning-Adapter'
  - 데이터셋: 'prepared_finetuning_dataset_splits_arrow'

⏳ 베이스 모델을 로딩합니다...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 베이스 모델 로딩 완료.
⏳ 베이스 모델에 'totalcream/Qwen3-4B-finetunning-Adapter' LoRA 어댑터를 적용합니다...


adapter_config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/66.1M [00:00<?, ?B/s]

✅ LoRA 어댑터 적용 완료.

✅ 최종 모델 준비 완료!


In [ ]:
# --- 데이터셋 전처리 ---

# 1. 데이터셋 로드
# 저장된 Arrow 형식의 데이터셋에서 'test' 스플릿을 불러옵니다.
try:
    dataset = load_from_disk(dataset_path)
    test_ds = dataset["test"]
    print("✅ 데이터셋 로딩 성공")
except FileNotFoundError:
    print(f"❌ 오류: '{dataset_path}' 경로에 데이터셋이 없습니다. 경로를 확인해주세요.")
    # 이 경우, 이후 셀은 실행되지 않습니다.

# 2. 프롬프트 템플릿의 각 섹션을 분리하는 함수
def split_sections(text: str):
    """'###' 헤더를 기준으로 텍스트를 Instruction, Input, Response로 분리합니다."""
    # 정규식 패턴: '### Instruction:', '### Input:', '### Response:'
    pattern = r"###\s*(Instruction|Input|Response)\s*:\s*"
    # re.split()을 사용하면 [헤더1, 내용1, 헤더2, 내용2, ...] 형태로 분리됩니다.
    parts = re.split(pattern, text)
    if len(parts) > 1:
        parts = parts[1:] # 맨 앞의 빈 문자열 제거
        # 딕셔너리 컴프리헨션을 사용하여 {헤더: 내용} 형태로 만듭니다.
        return {parts[i]: parts[i+1].strip() for i in range(0, len(parts), 2)}
    return {}

# 3. 데이터셋의 각 샘플을 파싱하여 컬럼 추가
def extract_sections_from_example(example):
    """데이터셋의 'text' 필드를 파싱하여 새로운 컬럼을 추가합니다."""
    # 기존 'text' 필드가 문자 리스트일 경우, 하나의 문자열로 합칩니다.
    full_text = "".join(example["text"])
    sections = split_sections(full_text)
    return {
        "instruction": sections.get("Instruction", ""),
        "input": sections.get("Input", ""),
        "response_gt": sections.get("Response", "") # Ground Truth (정답)
    }

# .map() 함수를 사용하여 모든 샘플에 대해 함수를 적용합니다.
test_ds = test_ds.map(extract_sections_from_example, batched=False)

print("\n✅ 데이터셋 전처리 완료")
print("새로운 컬럼:", test_ds.column_names)

# 전처리 결과 확인 (첫 번째 샘플)
print("\n--- 첫 번째 샘플 전처리 결과 ---")
print("Instruction:", test_ds[0]['instruction'])
print("Input:", test_ds[0]['input'])
print("Response (정답):", test_ds[0]['response_gt'])

✅ 데이터셋 로딩 성공

✅ 데이터셋 전처리 완료
새로운 컬럼: ['text', 'instruction', 'input', 'response_gt']

--- 첫 번째 샘플 전처리 결과 ---
Instruction: 다음은 [신호및시스템] 과목의 객관식 문제입니다. 문제와 보기를 읽고 정답 번호와 해설을 제공해주세요.
Input: 문제: 다음 중 단위 계단 신호 u(t)의 정의로 옳은 것은?
보기:
1: t>0에서 0
2: t>0에서 1
3: t<0에서 1
4: t=0에서 무한대
5: t<0에서 무한대
Response (정답): 정답: 2
해설: u(t)는 t>0에서 1이다.


In [ ]:
# --- ⭐⭐⭐ 수정된 핵심 평가 함수 ⭐⭐⭐ ---

def evaluate_with_logs(ds, model, tokenizer, batch_size=8):
    total, correct = 0, 0
    logs = []  # 결과 로깅을 위한 리스트

    # 데이터셋을 배치 단위로 처리
    for batch in tqdm(ds.batch(batch_size), desc="Evaluating"):
        # 평가를 위한 프롬프트 구성 (모델이 학습한 형식과 동일하게)
        prompts = [
            f"### Instruction:\n{inst}\n\n### Input:\n{inp}\n\n### Response:"
            for inst, inp in zip(batch["instruction"], batch["input"])
        ]

        # 프롬프트를 토큰화
        encodings = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        # 그래디언트 계산 비활성화 (추론 시 메모리 절약 및 속도 향상)
        with torch.no_grad():
            generated_ids = model.generate(
                **encodings,
                max_new_tokens=128,  # 모델이 충분한 답변을 생성하도록 길이 증가
                do_sample=False,     # 항상 동일한 결과를 얻기 위해 샘플링 비활성화
                pad_token_id=tokenizer.eos_token_id
            )

        # 생성된 토큰 ID를 다시 텍스트로 디코딩
        input_len = encodings.input_ids.shape[1]
        generated_texts = tokenizer.batch_decode(generated_ids[:, input_len:], skip_special_tokens=True)

        # 배치 내 각 샘플에 대해 채점
        for i, gen_text in enumerate(generated_texts):
            instruction = batch["instruction"][i]
            gt_text = batch["response_gt"][i]

            is_correct = False
            pred_answer = "N/A"  # 예측값 초기화
            gt_answer = "N/A"    # 정답값 초기화

            # === 문제 유형에 따라 평가 로직 분기 ===
            if "OX 퀴즈" in instruction:
                # [OX 퀴즈 평가]
                # 1. 정답(GT)에서 O 또는 X 추출 (대소문자 무시)
                m_gt = re.search(r'정답\s*[:：]\s*(O|X)', gt_text, re.IGNORECASE)
                if m_gt:
                    gt_answer = m_gt.group(1).upper()

                # 2. 모델 예측에서 맨 앞에 나오는 O 또는 X 추출
                m_pred = re.search(r'^\s*(O|X)', gen_text, re.IGNORECASE)
                if m_pred:
                    pred_answer = m_pred.group(1).upper()
            else:
                # [객관식 문제 평가]
                # 1. 정답(GT)에서 숫자 추출
                m_gt = re.search(r'정답\s*[:：]\s*(\d+)', gt_text)
                if m_gt:
                    gt_answer = int(m_gt.group(1))

                # 2. 모델 예측에서 '정답: <숫자>' 또는 '답: <숫자>' 패턴으로 숫자 추출
                m_pred = re.search(r'(?:정답|답)\s*[:：]?\s*(\d+)', gen_text)
                if m_pred:
                    pred_answer = int(m_pred.group(1))

            # === 채점 및 집계 ===
            # 유효한 정답이 있는 경우에만 채점
            if gt_answer != "N/A":
                is_correct = (pred_answer == gt_answer)
                if is_correct:
                    correct += 1
                total += 1

            # 분석을 위해 상세 로그 저장
            logs.append({
                "instruction": instruction.strip(),
                "gt_text": gt_text,
                "generated_text": gen_text.strip(),
                "gt_answer": gt_answer,
                "pred_answer": pred_answer,
                "match": is_correct
            })

    accuracy = (100 * correct / total) if total > 0 else 0
    df_logs = pd.DataFrame(logs)
    return accuracy, df_logs

# --- 평가 실행 ---
print("\n✅ 모델 평가를 시작합니다...")
accuracy, df_logs = evaluate_with_logs(test_ds, model, tokenizer, batch_size=batch_size)
print(f"\n✅ 평가 완료! 최종 정확도: {accuracy:.2f}%")


✅ 모델 평가를 시작합니다...


Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'


✅ 평가 완료! 최종 정확도: 36.36%


In [ ]:
# prompt: dp_logs를 csv로 저장하는 코드를 작성해 줘

# --- 4) 로그를 CSV 파일로 저장 ---
csv_filename = "dp_logs.csv"
df_logs.to_csv(csv_filename, index=False, encoding='utf-8-sig') # 한글 깨짐 방지를 위해 utf-8-sig 사용

print(f"\n✅ 평가 로그가 '{csv_filename}' 파일로 저장되었습니다.")

# 파일을 다운로드하려면 다음 줄의 주석을 해제하세요.
from google.colab import files
files.download(csv_filename)


✅ 평가 로그가 'dp_logs.csv' 파일로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files, output

# 1. dp_logs.csv 파일 업로드
print("👉 'dp_logs.csv' 파일을 업로드해주세요.")
uploaded = files.upload()

# 업로드된 파일명을 가져와서 DataFrame으로 로드
if uploaded:
    csv_filename = list(uploaded.keys())[0]
    df_logs = pd.read_csv(csv_filename)
    print(f"\n✅ '{csv_filename}' 파일을 성공적으로 불러왔습니다.")

    # 2. 오답 데이터만 필터링
    df_incorrect = df_logs[df_logs['match'] == False].copy()
    print(f"전체 로그 수: {len(df_logs)}, 오답 수: {len(df_incorrect)}")

    # 3. 수동 보정을 위한 새로운 컬럼 추가
    if 'manual_correction' not in df_logs.columns:
        df_logs['manual_correction'] = None # None으로 초기화

    print("\n✅ 오답 보정 작업 준비가 완료되었습니다. 다음 셀을 실행하여 보정을 시작하세요.")
else:
    print("\n❌ 파일이 업로드되지 않았습니다. 셀을 다시 실행하여 파일을 업로드해주세요.")

👉 'dp_logs.csv' 파일을 업로드해주세요.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- ipywidgets를 사용한 안정적이고 직관적인 UI ---

# 이전에 작업하던 내용이 있다면 초기화를 위해 전역 변수 선언
global current_index
current_index = -1 # -1에서 시작하여 첫 클릭 시 0이 되도록 함

# 오답 데이터의 인덱스 목록 생성
incorrect_indices = df_incorrect.index.tolist()
total_incorrect = len(incorrect_indices)

# --- 위젯 생성 (UI 요소들) ---
# 1. 텍스트 입력창
text_input = widgets.Text(
    value='',
    placeholder='여기에 보정값을 입력하세요...',
    description='보정값:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# 2. 버튼들
submit_button = widgets.Button(
    description="저장하고 다음",
    button_style='success', # 초록색
    tooltip='입력한 값을 저장하고 다음 항목으로 넘어갑니다.'
)
skip_button = widgets.Button(
    description="스킵하고 다음",
    button_style='info', # 파란색
    tooltip='저장하지 않고 다음 항목으로 넘어갑니다.'
)

# 3. 위젯들을 가로로 배치하기 위한 컨테이너
ui_container = widgets.HBox([text_input, submit_button, skip_button])


# --- 버튼 클릭 시 실행될 함수 정의 ---
def show_next_item(button_clicked):
    global current_index

    # --- 이전 단계의 입력값 저장 로직 ---
    # current_index가 0 이상일 때만(첫 실행이 아닐 때만) 저장 로직을 수행
    if current_index >= 0:
        # '저장하고 다음' 버튼이 눌렸고, 입력값이 있을 때만 저장
        if button_clicked.description == "저장하고 다음" and text_input.value:
            original_index = incorrect_indices[current_index]
            df_logs.loc[original_index, 'manual_correction'] = text_input.value

    # --- 다음 항목 표시 로직 ---
    current_index += 1 # 다음 인덱스로 이동

    # 모든 항목을 다 확인했으면 종료
    if current_index >= total_incorrect:
        clear_output()
        print("🎉 모든 오답 항목에 대한 보정 작업을 완료했습니다!")
        print("다음 셀을 실행하여 결과를 저장하세요.")
        return

    # 화면 정리 및 다음 항목 표시
    clear_output(wait=True)

    original_index = incorrect_indices[current_index]
    row = df_logs.loc[original_index]

    print(f"--- 🖥️  오답 보정 작업 ({current_index + 1}/{total_incorrect}) ---")
    if 'instruction' in row and pd.notna(row['instruction']):
        print(f"\n[지시문]\n{row['instruction']}\n")

    print("="*50)
    print(f"✅ 정답 (Ground Truth) : {row['gt_answer']}")
    print(f"❌ 모델 예측 (추출된 값) : {row['pred_answer']}")
    print(f"📄 모델이 생성한 전체 텍스트 :\n{row['generated_text']}")
    print("="*50)

    # 입력창 초기화 및 UI 다시 표시
    text_input.value = ''
    display(ui_container)

# --- 작업 시작 ---
if total_incorrect == 0:
    print("🎉 보정할 오답이 없습니다!")
else:
    # 각 버튼에 콜백 함수 연결
    submit_button.on_click(show_next_item)
    skip_button.on_click(show_next_item)

    # 최초 실행 시, UI를 먼저 표시하고 첫 번째 항목을 로드
    display(ui_container)
    # 임의의 버튼 객체를 전달하여 첫 번째 항목을 표시
    show_next_item(skip_button)

🎉 모든 오답 항목에 대한 보정 작업을 완료했습니다!
다음 셀을 실행하여 결과를 저장하세요.


In [ ]:
# 1. 보정 작업이 이루어진 행들만 확인
print("--- ✨ 보정 작업 결과 확인 ---")
corrected_rows = df_logs[df_logs['manual_correction'].notna()]

if corrected_rows.empty:
    print("보정된 항목이 없습니다.")
else:
    # 원본 정답, 모델 예측, 수동 보정값을 함께 출력하여 비교
    display(corrected_rows[['gt_answer', 'pred_answer', 'manual_correction', 'match']])

# 2. 보정된 전체 DataFrame을 새로운 CSV 파일로 저장
corrected_csv_filename = "corrected_logs.csv"
df_logs.to_csv(corrected_csv_filename, index=False, encoding='utf-8-sig') # 한글 깨짐 방지

print(f"\n✅ 보정된 전체 로그가 '{corrected_csv_filename}' 파일로 저장되었습니다.")

# 3. 보정된 파일 다운로드
files.download(corrected_csv_filename)

--- ✨ 보정 작업 결과 확인 ---


,gt_answer,pred_answer,manual_correction,match
1,O,NaN,O,False
2,4,NaN,4,False
4,X,NaN,X,False
5,X,NaN,X,False
6,5,NaN,5,False
...,...,...,...,...
135,X,O,X,False
137,O,NaN,O,False
139,X,O,X,False
141,O,NaN,O,False



✅ 보정된 전체 로그가 'corrected_logs.csv' 파일로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np

# --- 0. 데이터 준비 ---
# 이전에 'df_logs' 변수가 메모리에 있다고 가정합니다.
# 만약 없다면, 아래 주석을 해제하여 corrected_logs.csv에서 다시 불러오세요.
# try:
#     df_logs = pd.read_csv("corrected_logs.csv")
#     print("✅ 'corrected_logs.csv' 파일에서 데이터를 다시 불러왔습니다.")
# except FileNotFoundError:
#     print("❌ 'corrected_logs.csv' 파일을 찾을 수 없습니다. 이전 셀들을 먼저 실행해주세요.")


# --- 1. 최종 정답 여부(final_match) 컬럼 생성 ---

# 안전한 비교를 위해 관련 컬럼들을 문자열(string) 타입으로 변환
df_logs['gt_answer_str'] = df_logs['gt_answer'].astype(str)
df_logs['manual_correction_str'] = df_logs['manual_correction'].astype(str)

# 'manual_correction'이 비어있지 않고(notna), 실제 정답과 일치하는 경우를 찾음
successful_correction = (df_logs['manual_correction'].notna()) & \
                        (df_logs['gt_answer_str'] == df_logs['manual_correction_str'])

# 'final_match' 컬럼을 생성하고, 기본값은 원래 'match' 결과로 채움
df_logs['final_match'] = df_logs['match']

# 수동 보정에 성공한 경우, 'final_match' 값을 True로 덮어씀 (업데이트)
df_logs.loc[successful_correction, 'final_match'] = True


# --- 2. 보정 전/후 정확도 비교 ---

# 평가 가능한 전체 샘플 수 (정답이 'N/A'가 아닌 경우)
valid_logs = df_logs[df_logs['gt_answer'] != 'N/A'].copy()
total_evaluable = len(valid_logs)

# 보정 전/후 정답 수 계산
old_correct_count = valid_logs['match'].sum()
new_correct_count = valid_logs['final_match'].sum()

# 보정 전/후 정확도 계산
old_accuracy = (100 * old_correct_count / total_evaluable) if total_evaluable > 0 else 0
new_accuracy = (100 * new_correct_count / total_evaluable) if total_evaluable > 0 else 0

print("--- ✨ 최종 정확도 요약 (보정 결과 반영) ✨ ---")
summary_data = {
    '': ['정답 수', '전체 샘플 수', '정확도 (%)'],
    '보정 전': [old_correct_count, total_evaluable, f"{old_accuracy:.2f}%"],
    '보정 후': [new_correct_count, total_evaluable, f"{new_accuracy:.2f}%"],
    '개선': [f"+{new_correct_count - old_correct_count}개", "", f"+{new_accuracy - old_accuracy:.2f}%p"]
}
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))


# --- 3. 오답에서 정답으로 변경된 사례 확인 ---
print("\n--- ✅ 오답에서 정답으로 변경된 사례 ---")
flipped_to_correct = valid_logs[(valid_logs['match'] == False) & (valid_logs['final_match'] == True)]

if flipped_to_correct.empty:
    print("수동 보정을 통해 정답으로 변경된 항목이 없습니다.")
else:
    # 비교에 용이한 컬럼들만 선택하여 출력
    display(flipped_to_correct[['gt_answer', 'pred_answer', 'manual_correction', 'generated_text']].head(10))


# --- 4. 과목별 최종 정확도 분석 ---
print("\n--- 📊 과목별 최종 정확도 (보정 후) ---")
valid_logs['subject'] = valid_logs['instruction'].str.extract(r'\[(.+?)\]').fillna('기타')

final_subject_summary = (
    valid_logs
    .groupby('subject')['final_match'] # final_match 기준으로 집계
    .agg(total='count', correct='sum')
    .reset_index()
)
final_subject_summary['accuracy(%)'] = round(final_subject_summary['correct'] / final_subject_summary['total'] * 100, 2)
final_subject_summary = final_subject_summary.sort_values(by='accuracy(%)', ascending=False)

print(final_subject_summary.to_string(index=False))

--- ✨ 최종 정확도 요약 (보정 결과 반영) ✨ ---
          보정 전   보정 후       개선
   정답 수     52     86     +34개
전체 샘플 수    143    143         
정확도 (%) 36.36% 60.14% +23.78%p

--- ✅ 오답에서 정답으로 변경된 사례 ---


,gt_answer,pred_answer,manual_correction,generated_text
4,X,N/A,X,설명을 위해 간략히 설명해주세요.\n\n### Input:\n질문: RLC는 리지o...
5,X,N/A,X,보기:\n1: O\n2: X\n3: O\n4: X\n5: O\n\n### Instr...
10,2,N/A,2,"의정답은 2번이며, 해설은 ""단위 계단 신호의 라플라스 변환은 1/s이다.""입니다."
13,O,N/A,O,"정답: O\n해설: ""쉘 정렬은 삽입 정렬을 기본 배열 구조에 따라 초기화하고 교환..."
14,O,N/A,O,"문제: 0\n해설: ""인과적 LTI 시스템은 임펄스 응답 h[n]이 n<0일 때도 ..."
19,O,N/A,O,대답:\n정답: O\n해설: 이변수 변환의 공식으로 연속 확률변수의 경우 정의에 따...
20,X,N/A,X,방식:\n정답: X\n해설: RLC 직렬 공진 시 전압과 전류의 위상차는 0도이다.
27,O,N/A,O,ANDLE:\n정답: O\n해설: 유전체는 전기장이 가해져도 내부 전하가 이동하지 ...
33,3,N/A,3,"번호와 해설을 제공해주세요.\n\n### Input:\n문제: 이산 확률벡터(x,y..."
37,O,N/A,O,설명을 요청합니다.



--- 📊 과목별 최종 정확도 (보정 후) ---
  subject  total  correct  accuracy(%)
    멀티미디어      6        5        83.33
 디지털 논리회로      6        5        83.33
객체지향프로그래밍      5        4        80.00
    정보보호론      5        4        80.00
   기계학습개론     10        7        70.00
   디지털시스템      9        6        66.67
 데이터베이스설계     11        7        63.64
   신호및시스템     11        7        63.64
  컴퓨터네트워크     17       10        58.82
     회로이론     14        8        57.14
    자료구조론      6        3        50.00
    전자기학1     16        8        50.00
   알고리즘설계     15        7        46.67
     확률변수     12        5        41.67
